In [1]:
import os
import subprocess
from sklearn.preprocessing import StandardScaler
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem

# Generate Orca Inputs

In [9]:
def calculate_multiplicity(mol):
    """
    Calculate the multiplicity based on the number of unpaired (radical) electrons.
    
    Args:
    mol (rdkit.Chem.Mol): RDKit molecule object.
    
    Returns:
    multiplicity (int): The calculated multiplicity of the molecule.
    """
    n_radical_electrons = 0
    # Loop through atoms and sum the number of unpaired electrons
    for atom in mol.GetAtoms():
        n_radical_electrons += atom.GetNumRadicalElectrons()

    # Calculate multiplicity: Multiplicity = n_radical_electrons + 1
    if n_radical_electrons == 0:
        multiplicity = 1  # Singlet (all electrons paired)
    else:
        multiplicity = n_radical_electrons + 1  # Doublet, triplet, etc.
    
    return multiplicity

def determine_charge_and_multiplicity(mol):
    """
    Determines the formal charge and the multiplicity of the molecule.
    
    Args:
    mol (rdkit.Chem.Mol): RDKit molecule object.
    
    Returns:
    charge (int): The formal charge of the molecule.
    multiplicity (int): The calculated multiplicity based on the number of radical electrons.
    """
    # Determine the formal charge using RDKit's built-in function
    charge = Chem.GetFormalCharge(mol)

    # Calculate multiplicity using the custom function
    multiplicity = calculate_multiplicity(mol)

    return charge, multiplicity


def smiles_to_orca_input(smiles_string, chembl_id, output_inp_file, method="B3LYP", basis_set="6-31G*"):
    """
    Converts a SMILES string to a 3D structure, determines charge/multiplicity, and generates ORCA input file.
    
    Args:
    smiles_string (str): SMILES string representing the molecule.
    chembl_id (str): Identifier for the molecule (used in file naming).
    output_inp_file (str): The name of the ORCA input file to be generated.
    method (str): Quantum chemistry method (default is B3LYP).
    basis_set (str): Basis set (default is 6-31G*).
    """
    mol = Chem.MolFromSmiles(smiles_string)
    mol = Chem.AddHs(mol)  # Add hydrogens
    AllChem.EmbedMolecule(mol, AllChem.ETKDG())  # Generate 3D coordinates
    AllChem.UFFOptimizeMolecule(mol)  # Optimize geometry

    # Determine the charge and multiplicity
    charge, multiplicity = determine_charge_and_multiplicity(mol)

    # Generate ORCA input file content 
    orca_input = f"! {method} {basis_set} SP\n\n* xyz {charge} {multiplicity}\n"
    
    # Write atomic coordinates to the ORCA input file
    with open(output_inp_file, 'w') as inp_file:
        inp_file.write(orca_input)
        for atom in mol.GetAtoms():
            pos = mol.GetConformer().GetAtomPosition(atom.GetIdx())
            symbol = atom.GetSymbol()
            inp_file.write(f"{symbol} {pos.x:.4f} {pos.y:.4f} {pos.z:.4f}\n")
        inp_file.write("*\n")

    print(f"ORCA input file '{output_inp_file}' generated successfully with charge {charge} and multiplicity {multiplicity}.")


def process_csv(csv_file):
    """
    Processes a CSV file containing SMILES strings and ChEMBL IDs to generate ORCA input files.
    
    Args:
    csv_file (str): Path to the CSV file.
    """
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    # extract SMILES from the 6th column (index 5) and ChEMBL IDs from 1st column (index 0)
    for index, row in df.iterrows():
        chembl_id = row[0]  # ChEMBL ID from the first column
        smiles = row[5]  # SMILES from the 6th column
        
        # Define output filenames using the ChEMBL ID
        output_inp_file = f"{out_dir}/{chembl_id}.inp"
        
        print(f"Processing {chembl_id}: {smiles}")
        
        # Convert SMILES to ORCA input
        smiles_to_orca_input(smiles, chembl_id, output_inp_file)


In [11]:
#Process a CSV file
out_dir = "orca_inputs"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    print(f"Directory '{out_dir}' created.")
else:
    print(f"Directory '{out_dir}' already exists.")
    
csv_file = "../1_preprocess/TRPM8-homosapien-compounds-activities-processed.csv"  # Update this with the actual path to your CSV file


process_csv(csv_file)

Processing CHEMBL3235962: N#Cc1cccc(NC(=O)N2CCc3ccccc3[C@H]2c2ccc(C(F)(F)F)cc2)c1
ORCA input file 'CHEMBL3235962.inp' generated successfully with charge 0 and multiplicity 1.
Processing CHEMBL3235983: C[C@H](NC(=O)N1CCc2ccccc2[C@H]1c1ccc(C(F)(F)F)c(F)c1)C(F)(F)F
ORCA input file 'CHEMBL3235983.inp' generated successfully with charge 0 and multiplicity 1.
Processing CHEMBL1650511: FC(F)(F)c1ccccc1-c1cc(C(F)(F)F)c2[nH]c(C3=NOC4(CCCCC4)C3)nc2c1
ORCA input file 'CHEMBL1650511.inp' generated successfully with charge 0 and multiplicity 1.
Processing CHEMBL2443068: O=C1CC2(CCN(C(=O)Nc3ccc(C(F)(F)F)cc3)CC2)Oc2c(Cl)cccc21
ORCA input file 'CHEMBL2443068.inp' generated successfully with charge 0 and multiplicity 1.
Processing CHEMBL3959823: Cc1cccc(CN(C(=O)c2ccccc2)[C@@H](C(N)=O)c2ccccc2)c1
ORCA input file 'CHEMBL3959823.inp' generated successfully with charge 0 and multiplicity 1.
Processing CHEMBL3944840: C[C@H](c1cccc(Cl)c1)N(C(=O)c1ccccc1O)[C@@H](C(N)=O)c1ccccc1
ORCA input file 'CHEMBL3944840.

# Run Orca

In [ ]:
def run_orca_batch(input_folder,output_folder):
    """
    This function will search for all .inp files in the specified folder, 
    run ORCA on each, and generate corresponding .out files.
    """
    # List all files in the directory
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".inp"):
            # Get the base name without the .inp extension
            base_name = os.path.splitext(file_name)[0]
            inp_file = os.path.join(input_folder, file_name)
            out_file = os.path.join(input_folder, f"{base_name}.out")
            
            # Run ORCA and redirect output to the .out file
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)
                print(f"Directory '{output_folder}' created.")
            else:
                print(f"Directory '{output_folder}' already exists.")
            
            print(f"Processing: {file_name}")
            with open(f"{output_folder}/{out_file}", "w") as out_f:
                subprocess.run(["orca_path/orca_6_0_0/orca", inp_file], stdout=out_f)

# Example: Set the folder containing .inp files
input_folder = "orca_inputs/"  # Replace with the correct folder path
output_folder = "orca_outputs"
# Run the batch process
run_orca_batch(input_folder,output_folder)

# Extract Quantum Descriptors

In [ ]:
def extract_homo_lumo(orca_output_file):
    """
    Extracts the HOMO and LUMO energies from an ORCA output file.

    Args:
    orca_output_file (str): Path to the ORCA output file.

    Returns:
    tuple: HOMO and LUMO energies, or (None, None) if not found.
    """
    try:
        with open(orca_output_file, 'r') as f:
            lines = f.readlines()
        
        homo_energy = None
        lumo_energy = None
        orbital_energies = []
        start_index = None

        # Find the section that contains "NO   OCC          E(Eh)            E(eV)"
        for i, line in enumerate(lines):
            if "  NO   OCC          E(Eh)            E(eV)" in line:
                start_index = i
                break

        if start_index is None:
            print(f"No 'ORBITAL ENERGIES' section found in {orca_output_file}")
            return None, None

        # Extract orbital energies from the section
        for line in lines[start_index + 1:]:
            if "---" in line or line.strip() == "":  # End of section
                break

            parts = line.split()
            if len(parts) == 4:
                occ = float(parts[1])  # Occupation number
                energy = float(parts[2])  # Energy in Eh (Hartrees)
                orbital_energies.append((occ, energy))

        # Identify HOMO (last occupied orbital) and LUMO (first unoccupied orbital)
        for i, (occ, energy) in enumerate(orbital_energies):
            if occ == 2.00:
                homo_energy = energy
            if occ == 0.00 and lumo_energy is None:
                lumo_energy = energy

        return homo_energy, lumo_energy
    except Exception as e:
        print(f"Error processing file {orca_output_file}: {e}")
        return None, None

def calculate_quantum_descriptors(homo, lumo):
    """
    Calculates electronegativity and chemical hardness based on HOMO and LUMO energies.

    Args:
    homo (float): HOMO energy.
    lumo (float): LUMO energy.

    Returns:
    tuple: Electronegativity and chemical hardness.
    """
    if homo is None or lumo is None:
        return None, None

    # Electronegativity (χ) and Chemical Hardness (η)
    electronegativity = - (homo + lumo) / 2
    hardness = (lumo - homo) / 2

    return electronegativity, hardness

def process_folder_and_save_csv_as_new(folder_path, original_csv_file, updated_csv_file):
    """
    Processes all ORCA .out files in a given folder, extracts HOMO/LUMO, and saves to a new CSV file.

    Args:
    folder_path (str): Path to the folder containing .out files.
    original_csv_file (str): Path to the original CSV file.
    updated_csv_file (str): Path to save the updated CSV file.
    """
    # Load the CSV file
    df = pd.read_csv(original_csv_file)[['Molecule ChEMBL ID']]

    # use the first column as the identifier column
    identifier_column = df.columns[0]

    if 'HOMO' not in df.columns:
        df['HOMO'] = None
    if 'LUMO' not in df.columns:
        df['LUMO'] = None
    if 'Electronegativity' not in df.columns:
        df['Electronegativity'] = None
    if 'Hardness' not in df.columns:
        df['Hardness'] = None

    # Process each .out file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".out"):
            file_path = os.path.join(folder_path, filename)

            # Extract HOMO and LUMO energies from the .out file
            homo, lumo = extract_homo_lumo(file_path)

            if homo is not None and lumo is not None:
                # Calculate electronegativity and hardness
                electronegativity, hardness = calculate_quantum_descriptors(homo, lumo)

                # Use the first column to match the file
                file_id = os.path.splitext(filename)[0]

                # Update the CSV for the corresponding row
                df.loc[df[identifier_column] == file_id, 'HOMO'] = homo
                df.loc[df[identifier_column] == file_id, 'LUMO'] = lumo
                df.loc[df[identifier_column] == file_id, 'Electronegativity'] = electronegativity
                df.loc[df[identifier_column] == file_id, 'Hardness'] = hardness

    # Save the updated CSV file with a new name
    df.to_csv(updated_csv_file, index=False)
    print(f"Updated CSV file saved as {updated_csv_file}")

# Example usage:
folder_path = "orca_outputs/"  # Path to folder containing .out files
original_csv_file = "../1_preprocess/1_TRPM8-homosapien-compounds-activities-processed.csv"  # Path to your original CSV file
updated_csv_file = "quantum-descriptors.csv"  # Path to save the updated CSV
process_folder_and_save_csv_as_new(folder_path, original_csv_file, updated_csv_file)

In [ ]:
# Load the CSV file
df = pd.read_csv("quantum-descriptors.csv")

# Select the quantum descriptor columns for standardization
quantum_columns = ['HOMO', 'LUMO', 'Electronegativity', 'Hardness']

scaler = StandardScaler()
df[quantum_columns] = scaler.fit_transform(df[quantum_columns])

df.to_csv("quantum-descriptors-standardized.csv", index=False)
print("Standardized data saved as standardized_updated.csv")


# Find Mean Absolute Deviation (MAD) and variance and append to CSV

In [ ]:
import os
import numpy as np
import pandas as pd

def process_orca_outputs_and_append_to_csv(folder_path, csv_file_path, output_csv_file_path):
    # Load the existing CSV file
    df = pd.read_csv(csv_file_path)
    
    # Create new columns for Variance and MAD if they don't exist
    if 'Variance' not in df.columns:
        df['Variance'] = np.nan  
    if 'MAD' not in df.columns:
        df['MAD'] = np.nan  
    # Loop through all ORCA output files in the specified folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.out'):
            file_path = os.path.join(folder_path, filename)
            
            with open(file_path, 'r') as file:
                lines = file.readlines()
                collect_charges = False
                mulliken_charges = []  # store Mulliken charges
                
                # Find and process the Mulliken atomic charges section
                for i, line in enumerate(lines):
                    if 'MULLIKEN ATOMIC CHARGES' in line:
                        collect_charges = True
                        continue  # Move to the next line after finding the section
                    elif collect_charges:
                        # Stop collecting charges when you reach "Sum of atomic charges"
                        if 'Sum of atomic charges' in line:
                            break
                        # Extract charges following the ":" symbol
                        try:
                            split_line = line.split(':')
                            if len(split_line) == 2:
                                charge = float(split_line[1].strip().split()[0])
                                mulliken_charges.append(charge)
                        except ValueError:
                            continue  # Skip lines that do not match the format
                
                # Calculate variance and MAD if charges were found
                if mulliken_charges:
                    variance_charge = np.var(mulliken_charges)
                    mad_charge = np.mean(np.abs(mulliken_charges - np.mean(mulliken_charges)))
                    
                    # Append the variance and MAD to the CSV DataFrame based on file name match
                    file_name_without_ext = filename.replace('.out', '')
                    # Locate the matching row based on the filename in the first column of the existing csv
                    if file_name_without_ext in df.iloc[:, 0].values:
                        df.loc[df.iloc[:, 0] == file_name_without_ext, 'Variance'] = variance_charge
                        df.loc[df.iloc[:, 0] == file_name_without_ext, 'MAD'] = mad_charge
                    else:
                        print(f"File name {file_name_without_ext} not found in CSV file.")
    
    # Save the updated DataFrame to a new CSV file
    df.to_csv(output_csv_file_path, index=False)
    print(f"Updated CSV file saved as {output_csv_file_path}")

# Example usage:
folder_path = 'orca_outputs/'  # Replace with the path to your ORCA output files
csv_file_path = 'quantum-descriptors-standardized.csv'  # Replace with the path to your existing CSV file
output_csv_file_path = 'quantum-descriptors-standardized_updated_wParital_charges.csv'  # Replace with the path to save the new CSV file

process_orca_outputs_and_append_to_csv(folder_path, csv_file_path, output_csv_file_path)
